In [2]:
import json
import numpy as np
import os
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

In [3]:
df = np.load('../qm9_filtered.npy', allow_pickle = True)

In [4]:
df_X_0 = []
df_y_0 = []
df_X_1 = []
df_y_1 = []
df_X_2 = []
df_y_2 = []
df_X_3 = []
df_y_3 = []
df_X_4 = []
df_y_4 = []

for line in df:
    num = len(line['chiral_centers'])
    
    if num > 4:
        continue
    elif num == 0:
        df_X_0.append(line['xyz'].flatten())
        df_y_0.append(0)
    elif num == 1:
        df_X_1.append(line['xyz'].flatten())
        df_y_1.append(1)
    elif num == 2:
        df_X_2.append(line['xyz'].flatten())
        df_y_2.append(2)
    elif num == 3:
        df_X_3.append(line['xyz'].flatten())
        df_y_3.append(3)
    elif num == 4:
        df_X_4.append(line['xyz'].flatten())
        df_y_4.append(4)
    
df_X_0 = np.array(df_X_0)
df_y_0 = np.array(df_y_0)
df_X_1 = np.array(df_X_1)
df_y_1 = np.array(df_y_1)
df_X_2 = np.array(df_X_2)
df_y_2 = np.array(df_y_2)
df_X_3 = np.array(df_X_3)
df_y_3 = np.array(df_y_3)
df_X_4 = np.array(df_X_4)
df_y_4 = np.array(df_y_4)
    
idx_0 = np.random.choice(np.arange(len(df_y_0)), 10000, replace=False)
df_X_0 = df_X_0[idx_0]
df_y_0 = df_y_0[idx_0]

idx_1 = np.random.choice(np.arange(len(df_y_1)), 10000, replace=False)
df_X_1 = df_X_1[idx_1]
df_y_1 = df_y_1[idx_1]

idx_2 = np.random.choice(np.arange(len(df_y_2)), 10000, replace=False)
df_X_2 = df_X_2[idx_2]
df_y_2 = df_y_2[idx_2]

idx_3 = np.random.choice(np.arange(len(df_y_3)), 10000, replace=False)
df_X_3 = df_X_3[idx_3]
df_y_3 = df_y_3[idx_3]

idx_4 = np.random.choice(np.arange(len(df_y_4)), 10000, replace=False)
df_X_4 = df_X_4[idx_4]
df_y_4 = df_y_4[idx_4]


df_X = np.concatenate((df_X_0, df_X_1, df_X_2, df_X_3, df_X_4), axis=0)
df_y = np.concatenate((df_y_0, df_y_1, df_y_2, df_y_3, df_y_4), axis=0)

In [5]:
print(len(df_X), len(df_y))

50000 50000


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size = 0.2)

In [9]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

rfc = RandomForestClassifier()
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 10, num = 5)]

# number of features at every split
max_features = ['auto', 'sqrt']

# max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)

# create random grid
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_leaf': min_samples_leaf
 }

# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the model
rfc_random.fit(X_train, y_train)

# print results
print(rfc_random.best_params_)

NameError: name 'RandomForestClassifier' is not defined

In [13]:
from xgboost import XGBClassifier

clf = XGBClassifier(max_depth=20)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

y_train_pred = clf.predict(X_train)

print("Train acc: ", accuracy_score(y_train, y_train_pred))
print("test acc: ", accuracy_score(y_test, y_pred))
print("f1: ", f1_score(y_test, y_pred, average = 'weighted'))

/home/yzhou76/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:41:19] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train acc:  1.0
test acc:  0.7627
f1:  0.7621553036095737


In [14]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[1731,  183,   70,   36,    6],
       [ 249, 1397,  220,   73,   15],
       [  64,  293, 1293,  236,   82],
       [  15,   77,  225, 1503,  220],
       [   8,   20,   79,  202, 1703]])